# Load Tasks and VASP Files From NERSC

### Load Imports And Open Maggma Stores

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
%run imports.py

/home/ayush/miniconda3/envs/scan/lib/python3.8/site-packages/pymatgen/ext/matproj.py:454: DeprecationWarning:

__init__ is deprecated
MaterialsProjectCompatibility will be updated with new correction classes as well as new values of corrections and uncertainties in 2020



Imports successfully loaded


In [4]:
working_dir = os.getcwd()

In [5]:
%run maggma_stores.py

All maggma stores (db, elfcar_store, chgcar_store, aeccar0_store, aeccar2_store) successfully loaded 
Run 'connect_to_stores()' to connect to stores


In [6]:
connect_to_stores()

All connections successful


### Retrieve Tasks From NERSC

In [7]:
tasks_iter = db.query({'tags':{'$regex':'production-scan'}})
tasks = [t for t in tasks_iter]
len(tasks)

8295

### Sort Element and Binary Compound Tasks

In [8]:
is_binary = lambda f: len(Composition(f).elements) == 2
binary_tasks = [t for t in tasks if is_binary(t['formula_pretty'])]
len(binary_tasks)

5726

In [9]:
tasks_dict = {}

for t in binary_tasks:
    f = t['formula_pretty']
    if f not in tasks_dict.keys():
        tasks_dict[f] = t
    elif t['output']['energy_per_atom'] < tasks_dict[f]['output']['energy_per_atom']:
        tasks_dict[f] = t

In [10]:
is_element = lambda f: len(Composition(f).elements) == 1
element_tasks = [t for t in tasks if is_element(t['formula_pretty'])]
len(element_tasks)

712

In [11]:
elem_tasks_dict = {}

for t in element_tasks:
    f = re.sub(r'\d+', '', t['formula_pretty'])
    if f not in elem_tasks_dict.keys():
        elem_tasks_dict[f] = t
    elif t['output']['energy_per_atom'] < elem_tasks_dict[f]['output']['energy_per_atom']:
        elem_tasks_dict[f] = t

In [12]:
elem_data_json = {f:elem_tasks_dict[f] for f in elem_tasks_dict.keys()}
elem_ids = {f:elem_data_json[f]['task_id'] for f in elem_data_json.keys()}

### Find Subset of Binary Compounds to Study 
#### Formula must: (1) be in SCAN calculations, (2) be in JANAF experimental data, (3) match specified anion/cation criteria 

In [13]:
scan_formulas = set([Composition(t['formula_pretty']) for t in binary_tasks])
len(scan_formulas)

4943

In [15]:
janaf_formulas = []
janaf_data = pd.read_csv('data/janaf_thermo.csv')

for f in janaf_data['Formula'].tolist():
    try:
        janaf_formulas.append(Composition(f))
    except:
        print(f)
        
janaf_formulas = set(janaf_formulas)

e-


In [16]:
mp_expt_data = loadfn('data/MP_expt_energy_all.json')
mp_expt_formulas = set([Composition(f) for f in mp_expt_data.keys()])

In [17]:
len(scan_formulas & (mp_expt_formulas | janaf_formulas))

626

In [18]:
task_ids = [t['task_id'] for t in tasks_dict.values()]

In [19]:
anions = set([Element(e) for e in ['N', 'O', 'F', 'Cl', 'Br']])
cats = ['Ti', 'V', 'Cr', 'Mn', 'Fe', 'Ni'] + ['Li', 'Na', 'K', 'Rb', 'Cs'] + ['Be', 'Mg', 'Ca', 'Sr', 'Ba']
cations = set([Element(e) for e in cats])

In [20]:
selection = [f for f in (scan_formulas & (mp_expt_formulas | janaf_formulas)) if \
             set(Composition(f).elements) & anions or set(Composition(f).elements) & cations]
len(selection)

427

In [21]:
sel = [f for f in (scan_formulas & (mp_expt_formulas | janaf_formulas)) if \
             set(Composition(f).elements) & anions and set(Composition(f).elements) & cations]
len(sel)

96

In [22]:
sel

[Comp: Be3 N2,
 Comp: Be1 F2,
 Comp: Be1 O1,
 Comp: Li3 N1,
 Comp: Li2 O1,
 Comp: Be1 Cl2,
 Comp: Ca3 N2,
 Comp: Ca1 O1,
 Comp: K1 F1,
 Comp: K2 O1,
 Comp: K2 O2,
 Comp: Ca1 F2,
 Comp: Be1 Br2,
 Comp: Ca1 Br2,
 Comp: K1 Br1,
 Comp: Ba1 F2,
 Comp: Li1 F1,
 Comp: Ba1 Cl2,
 Comp: Ba1 Br2,
 Comp: Mg1 O1,
 Comp: Mg1 F2,
 Comp: K1 O2,
 Comp: Na1 Cl1,
 Comp: Mg1 Cl2,
 Comp: Cr2 N1,
 Comp: Cr1 N1,
 Comp: Cr2 O3,
 Comp: V2 O3,
 Comp: V1 O2,
 Comp: Cr1 F2,
 Comp: Cr1 F3,
 Comp: V1 F4,
 Comp: Fe2 O3,
 Comp: Fe3 O4,
 Comp: Fe1 F2,
 Comp: Mn1 F2,
 Comp: Fe1 F3,
 Comp: K1 Cl1,
 Comp: Ni1 O1,
 Comp: Ca1 Cl2,
 Comp: Li1 Br1,
 Comp: V1 Cl2,
 Comp: V1 Cl3,
 Comp: V1 Cl4,
 Comp: Cr1 Cl3,
 Comp: Mn1 Cl2,
 Comp: Fe1 Cl2,
 Comp: Fe1 Cl3,
 Comp: Ni1 Cl2,
 Comp: Rb2 O1,
 Comp: Na1 Br1,
 Comp: Mg1 Br2,
 Comp: Rb1 F1,
 Comp: Sr1 O1,
 Comp: Rb1 Cl1,
 Comp: Sr1 Cl2,
 Comp: Li2 O2,
 Comp: Ti1 Br3,
 Comp: Ti1 Br4,
 Comp: Mn1 Br2,
 Comp: Fe1 Br2,
 Comp: Cs2 O1,
 Comp: Cs1 O2,
 Comp: Ba1 O1,
 Comp: Ba1 O2,
 Comp: Cs1

In [20]:
selected_formulas = [c.reduced_formula for c in selection]
print(selected_formulas)

['LiH', 'BeH2', 'H2O', 'Be2C', 'Be3N2', 'BN', 'B2O3', 'BeF2', 'BeO', 'Li3N', 'Li2O', 'AlN', 'Al2O3', 'AlF3', 'BeCl2', 'BeS', 'CaH2', 'AlCl3', 'CCl4', 'Ca3N2', 'CaO', 'KF', 'K2O', 'K2O2', 'CaF2', 'Cr5B3', 'BeBr2', 'BBr3', 'As2O3', 'AsF3', 'As2O5', 'CoO', 'CaS', 'AlBr3', 'AsCl3', 'CoCl2', 'Ag2O', 'AgF', 'BaH2', 'BeI2', 'CaBr2', 'Rb2S', 'CaSe', 'KBr', 'AgCl', 'BaF2', 'LiF', 'CoBr2', 'BaS', 'BaCl2', 'CdCl2', 'Ce2O3', 'CeF3', 'NdF3', 'EuN', 'Ca3Sb2', 'EuO', 'AgBr', 'CeCl3', 'Er2O3', 'Au2O3', 'EuCl3', 'BaBr2', 'BiF3', 'Bi2O3', 'ErCl3', 'MgB2', 'AuCl', 'AuCl3', 'HfCl4', 'BiCl3', 'Ca2Pb', 'Ba2Sn', 'MgO', 'AuBr', 'MgF2', 'HfBr4', 'BiBr3', 'SiO2', 'HgBr', 'PbBr2', 'UFe2', 'SO3', 'ThBr4', 'UBr4', 'Mg2Si', 'KO2', 'Na2S', 'CrB', 'MgS', 'NaCl', 'ScN', 'TiC', 'Cr23C6', 'Cr7C3', 'Cr3C2', 'MgCl2', 'Cr2N', 'CrN', 'Fe2B', 'FeB', 'Mn2B', 'CaMg2', 'Cr2O3', 'V2O3', 'VO2', 'PCl3', 'CaAl2', 'CrF2', 'CrF3', 'VF4', 'Ni3B', 'CaSi', 'Fe2O3', 'Fe3O4', 'Ni2B', 'Ni4B3', 'Co3O4', 'FeF2', 'K2S', 'MnF2', 'FeF3', 'CoF3'

In [21]:
selected_ids = [tasks_dict[f]['task_id'] for f in selected_formulas]
print(selected_ids)

[6405, 5600, 8356, 3890, 7373, 11133, 5804, 10090, 3914, 3957, 3947, 3955, 4648, 4827, 6017, 3901, 5450, 6454, 11002, 9634, 3926, 3992, 4311, 5692, 4019, 6110, 6240, 7703, 8519, 7365, 8363, 4617, 4079, 8414, 8172, 10118, 4832, 3976, 6262, 10408, 6359, 5058, 4155, 4406, 3950, 2072, 2597, 4700, 4226, 4682, 5193, 10746, 9102, 8247, 4202, 11163, 4153, 4251, 7003, 9583, 6155, 8509, 7722, 7034, 10596, 7731, 3996, 11143, 10598, 7879, 10545, 10580, 8466, 3915, 6450, 4611, 10270, 11176, 5557, 5688, 6417, 6360, 7356, 8186, 8884, 4283, 4826, 4252, 2345, 3942, 3930, 3923, 3929, 8986, 9927, 5874, 6412, 7468, 2820, 5540, 6295, 6341, 11185, 5312, 7408, 2840, 8234, 5236, 5614, 7476, 7494, 8436, 4896, 10792, 11087, 4742, 9288, 6862, 4757, 2085, 6101, 6094, 7239, 4346, 4255, 6430, 5390, 5863, 4493, 6872, 4025, 4923, 4618, 5483, 5822, 4285, 4644, 5695, 4027, 4159, 4046, 7032, 4861, 5689, 6013, 4520, 6994, 4825, 9303, 9776, 8531, 5364, 4870, 5111, 7394, 10278, 4249, 4800, 7019, 9915, 6111, 6014, 5298, 693

In [22]:
binary_ids = {selected_formulas[i]:selected_ids[i] for i in range(len(selected_formulas))}

### Load ELFCARs, CHGCARs, and AECCARs (0 and 2)

In [25]:
# Binary Compounds

car_dict = {}

for ID in tqdm(selected_ids):
    elfcar = [e for e in elfcar_store.query({"metadata.task_id":ID})][0]
    if elfcar.get("data_aug"):
        del elfcar["data_aug"]
    ELF = Elfcar.from_dict(elfcar)
    
    chgcar = [e for e in chgcar_store.query({"metadata.task_id":ID})][0]
    CHG = Chgcar.from_dict(chgcar)

    aec0 = [e for e in aeccar0_store.query({"metadata.task_id":ID})][0]
    if aec0.get("data_aug"):
        del aec0["data_aug"]
    AEC0 = Chgcar.from_dict(aec0)

    aec2 = [e for e in aeccar2_store.query({"metadata.task_id":ID})][0]
    if aec2.get("data_aug"):
        del aec2["data_aug"] # bug fix line
    AEC2 = Chgcar.from_dict(aec2)
    
    car_dict[ID] = [ELF, CHG, AEC0, AEC2]


 71%|███████▏  | 305/427 [1:05:56<41:52, 20.60s/it]


100%|██████████| 427/427 [1:52:39<00:00, 15.83s/it]


In [26]:
elem_elfcars = {}

for ID in tqdm(elem_ids.values()):
    elfcar = [e for e in elfcar_store.query({"metadata.task_id":ID})][0]
    if elfcar.get("data_aug"):
        del elfcar["data_aug"]
    ELF = Elfcar.from_dict(elfcar)
    elem_elfcars[ID] = ELF


100%|██████████| 88/88 [00:18<00:00,  4.68it/s]


### Write Files Locally

In [27]:
# Binary Compounds

for i in tqdm(car_dict.keys()):
    label = str(i)
    lst = car_dict[i]
    filepath = 'files/'
    lst[0].write_file(filepath + 'ELFCAR_{0}'.format(str(i)))
    lst[1].write_file(filepath + 'CHGCAR_{0}'.format(str(i)))
    try:
        chgref = lst[2] + lst[3]
        chgref.write_file(filepath + 'CHGREF_{}'.format(str(i)))
    except ValueError:
        pass
    
dumpfn(binary_ids, 'data/binary_formulas_ids.json')


  0%|          | 0/427 [00:00<?, ?it/s]/home/ayush/miniconda3/envs/scan/lib/python3.8/site-packages/pymatgen/io/vasp/outputs.py:3010: UserWarning:

Structures are different. Make sure you know what you are doing...


 69%|██████▉   | 294/427 [1:51:35<24:58, 11.27s/it]


100%|██████████| 427/427 [2:27:59<00:00, 20.80s/it]


In [28]:
# Elements

for i in tqdm(elem_ids.values()):
    elfcar = elem_elfcars[i]
    filepath = 'files/'
    elfcar.write_file(filepath + 'ELFCAR_{0}'.format(str(i)))
    
dumpfn(elem_ids, 'data/element_formulas_ids.json')


 20%|██        | 18/88 [00:06<00:21,  3.25it/s]/home/ayush/miniconda3/envs/scan/lib/python3.8/site-packages/pymatgen/core/periodic_table.py:438: UserWarning:

No electronegativity for Ne. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.


 39%|███▊      | 34/88 [00:15<00:24,  2.16it/s]/home/ayush/miniconda3/envs/scan/lib/python3.8/site-packages/pymatgen/core/periodic_table.py:438: UserWarning:

No electronegativity for Ar. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.


100%|██████████| 88/88 [00:50<00:00,  1.73it/s]


In [29]:
# Tasks

task_ids_dict = {}
for t in tasks:
    task_ids_dict[t['task_id']] = t
    
def entry_from_task(task):
    return ComputedStructureEntry(Structure.from_dict(task["output"]["structure"]),
                                  task["output"]["energy"],
                                  parameters={"run_type": task["calcs_reversed"][0]["run_type"],
                                              "potcar_spec": task["calcs_reversed"][0]["input"]["potcar_spec"]},
                                  entry_id = task.get("task_id", None))

task_ids_selected = {}
for ID in list(car_dict.keys()) + list(elem_ids.values()):
    task_ids_selected[ID] = entry_from_task(task_ids_dict[ID])
    
dumpfn(task_ids_selected, 'data/computed_entries.json')

In [ ]:
# Experimental Energies
selected_expt_energies = {}

for i in range(len(selected_ids)):
    f = selected_formulas[i]
    if Composition(f) in janaf_formulas:
        selected_expt_energies[f] = janaf_data.iloc[janaf_data['Formula'] == f]['']
   

dumpfn(xxx, 'selected_experimental_energies.json')

In [15]:
ID = 7356

In [22]:
elfcar = [e for e in elfcar_store.query({"metadata.task_id":ID})][0]
if elfcar.get("data_aug"):
    del elfcar["data_aug"]
ELF = Elfcar.from_dict(elfcar)

In [23]:
chgcar = [e for e in chgcar_store.query({"metadata.task_id":ID})][0]
CHG = Chgcar.from_dict(chgcar)

In [24]:
aec0 = [e for e in aeccar0_store.query({"metadata.task_id":ID})][0]
if aec0.get("data_aug"):
    del aec0["data_aug"]
AEC0 = Chgcar.from_dict(aec0)

aec2 = [e for e in aeccar2_store.query({"metadata.task_id":ID})][0]
if aec2.get("data_aug"):
    del aec2["data_aug"] # bug fix line
AEC2 = Chgcar.from_dict(aec2)

In [28]:
ELF.write_file('ELFCAR_7356')
CHG.write_file('CHGCAR_7356')
AEC0.as_dict('AECCAR0_7356')
AEC2.write_file('AECCAR2_7356')

ValueError: invalid literal for int() with base 10: ''

In [31]:
dumpfn(AEC0.as_dict(), 'AECCAR0_7356.json')
dumpfn(AEC2.as_dict(), 'AECCAR2_7356.json')

In [33]:
AEC2.structure.composition

Comp: S4 O12